In [2]:
import numpy as np
from scipy.stats import geom, multivariate_normal

In [8]:
dim_x = 20
dim_z = 20
A = np.random.randn(dim_z, dim_x) #size = dim_z, dim_x
b = np.random.randn(dim_z) #size = dim_z, 1

def q_phi_z_given_x(x):
    mu = A @ x + b
    covariance = 2 / 3 * np.eye(dim_z)
    return multivariate_normal(mean=mu, cov=covariance)

def p_theta_x(theta):
    mu = theta
    covariance = 2 * np.eye(dim_x)
    return multivariate_normal(mean=mu, cov=covariance)

def p_theta_x_given_z(z):
    mu = z
    covariance = np.eye(dim_x)
    return multivariate_normal(mean=mu, cov=covariance)

def p_theta_z(theta):
    mu = theta
    covariance = np.eye(dim_z)
    return multivariate_normal(mean=mu, cov=covariance)

p = geom
r = 0.6

def logmeanexp(data, axis=None):
    max_val = np.max(data, axis=axis)
    return max_val + np.log(np.mean(np.exp(data - max_val), axis=axis))

In [9]:
K = p.rvs(r)
size = 2 ** (K + 1)
print(K)

2


In [10]:
theta_0 = [8] * dim_x
theta_0 = np.array(theta_0)
X = p_theta_x(theta_0).rvs(size=size)
print(X[:2])
print(X.shape)

[[6.31808303 8.77390846 4.92366596 8.98551383 9.57757736 7.98199988
  9.22903095 7.60030339 8.27494886 6.82415486 9.56223437 7.87599692
  7.58798954 7.01971754 8.56033027 4.84650841 8.67616978 8.71427736
  7.22697844 5.90048991]
 [6.58072881 8.52310897 6.49357977 8.05206245 7.49476588 6.54311708
  8.73654386 9.58364853 6.9224876  9.58189245 7.41368046 7.55969776
  7.76921688 5.36654986 8.35403714 9.40126566 7.52888284 5.58893039
  7.02168903 8.01527305]]
(8, 20)


In [19]:
Z = [q_phi_z_given_x(x).rvs() for x in X]
Z_E, Z_O = Z[::2], Z[1::2]
#weights = [p_theta_z(theta_0).pdf(z) * p_theta_x_given_z(z).pdf(x) / q_phi_z_given_x(x).pdf(z) for z in Z] si un seul x
log_weights = np.zeros(size)
for i in range(size):
    z = Z[i]
    x = X[i]
    log_weights[i] = np.log(p_theta_z(theta_0).pdf(z) * p_theta_x_given_z(z).pdf(x) / q_phi_z_given_x(x).pdf(z))

log_weights_E, log_weights_O = log_weights[::2], log_weights[1::2]
I_0 = np.mean(log_weights)


/tmp/ipykernel_32555/2516158896.py:8: RuntimeWarning: divide by zero encountered in log
  log_weights[i] = np.log(p_theta_z(theta_0).pdf(z) * p_theta_x_given_z(z).pdf(x) / q_phi_z_given_x(x).pdf(z))


In [20]:
l_hat_E, l_hat_O = logmeanexp(log_weights_E), logmeanexp(log_weights_O)
l_hat_O_E = logmeanexp(log_weights)

delta_K = l_hat_O_E - 0.5 * (l_hat_O + l_hat_E)

l_hat_ml_ss = I_0 + delta_K / p(r).pmf(K)

/tmp/ipykernel_32555/3376569390.py:31: RuntimeWarning: invalid value encountered in subtract
  return max_val + np.log(np.mean(np.exp(data - max_val), axis=axis))


In [21]:
delta_k_list = [logmeanexp(log_weights[:2**(k+1)]) - 0.5 * (logmeanexp(log_weights_O[:2**k]) + logmeanexp(log_weights_E[:2**k])) for k in range(K+1)]
l_hat_ml_rr = I_0 + np.sum([delta_k_list[k] / p(r).pmf(k) for k in range(K+1)])

/tmp/ipykernel_32555/3376569390.py:31: RuntimeWarning: invalid value encountered in subtract
  return max_val + np.log(np.mean(np.exp(data - max_val), axis=axis))
